In [1]:
import pandas as pd
from datetime import datetime, timedelta
from random import randint

### Функция для решения задачи

In [2]:
def task(order_df, city_df):
    # получаем id Москвы
    #city_id = city_df.query('city_name=="Москва"')['city_id'].item()
    """
    на случай, если названию городу может соответсвовать несколько id,
    например, автомобильный код региона
    """
    city_id = city_df.query('city_name=="Москва"')['city_id'].values
    # группируем заказы в Москве по пользователю и считаем выручку и количество заказов
    result_df = order_df[order_df.city_id.isin(city_id)].groupby(
        'user_id').agg({
            'order_id': 'count',
            'revenue': 'sum'
        })
    # переименовываем результирующие столбцы
    result_df.columns = ['orders_count', 'revenue_sum']

    return result_df

### Пример

In [3]:
city = pd.DataFrame({
    'city_id': [77, 78, 61],
    'city_name': ['Москва', 'Санкт-Петербург', 'Ростов-на-Дону']
})

city

,city_id,city_name
0,77,Москва
1,78,Санкт-Петербург
2,61,Ростов-на-Дону


In [4]:
now_ = datetime.now()

order = pd.DataFrame({
    'user_id': [1, 2, 3, 4, 2, 2, 2, 3, 4],
    'created_at':
    [now_ - timedelta(minutes=randint(61, 120)) for _ in range(9)],
    'finished_at':
    [now_ - timedelta(minutes=randint(1, 60)) for _ in range(9)],
    'city_id': [77, 78, 61, 77, 78, 78, 78, 61, 77],
    'revenue': [100 * i for i in range(1, 10)]
})
# сортиуем по времени создания
order = order.sort_values(by='created_at').reset_index(drop=True)
# индекс поездки - порядковый номер в отсортированном датафрейме
order = order.reset_index().rename(columns={'index': 'order_id'})
order

,order_id,user_id,created_at,finished_at,city_id,revenue
0,0,3,2023-03-07 16:59:32.654248,2023-03-07 18:27:32.654248,61,800
1,1,2,2023-03-07 17:00:32.654248,2023-03-07 18:23:32.654248,78,200
2,2,4,2023-03-07 17:21:32.654248,2023-03-07 18:01:32.654248,77,900
3,3,2,2023-03-07 17:28:32.654248,2023-03-07 18:03:32.654248,78,600
4,4,1,2023-03-07 17:41:32.654248,2023-03-07 18:35:32.654248,77,100
5,5,4,2023-03-07 17:45:32.654248,2023-03-07 18:53:32.654248,77,400
6,6,3,2023-03-07 17:46:32.654248,2023-03-07 18:35:32.654248,61,300
7,7,2,2023-03-07 17:52:32.654248,2023-03-07 18:52:32.654248,78,700
8,8,2,2023-03-07 17:54:32.654248,2023-03-07 18:51:32.654248,78,500


In [5]:
res = task(order, city)

assert (all(res['orders_count'] == [1, 2]))
assert (all(res['revenue_sum'] == [100, 1300]))

res

,orders_count,revenue_sum
user_id,,
1,1,100
4,2,1300
